In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain.schema.runnable import RunnableLambda

model = OllamaLLM(model="gpt-oss:20b")

In [2]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回復客戶郵件。請用友善、專業的語調回復。請不要回覆(公司的訊息和結尾詞)，我會自己手動增加"),
    ("human","請回覆這封客戶郵件 : \n\n{email_content}")
])

In [3]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("中華民國,西元%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    print(f"ai回覆的內容:{formatted_reply}")

    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)

In [4]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [5]:
chain = (prompt_template 
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
)

In [6]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。


王小明"""

In [7]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)

ai回覆的內容:您好王小明先生/女士，

感謝您選購本公司產品，也很抱歉得知您收到的商品有包裝損壞且功能與說明書不符。  
為了能盡快協助您解決問題，請您按下列步驟操作：

1. **拍照證明**  
   - 先拍攝包裝外觀、商品瑕疵及功能不符之處，並保留說明書照片。  
   - 這些圖片將作為退換貨的依據，請一併提供。

2. **填寫退換貨申請表**  
   - 前往我們的客服專區（[客服專區連結]），下載「退換貨申請表」並填寫。  
   - 在表格中請選擇「退貨」或「換貨」並說明原因。

3. **寄回商品**  
   - 我們將提供您一張免運費退換貨郵寄單。  
   - 請確保商品未拆封，保留原包裝並重新封裝。  
   - 將商品寄回後，請在郵件中附上申請表的掃描件或照片，並提供追蹤編號。

4. **核實與退款/換貨**  
   - 收到商品並進行檢查後，我們會在3–5個工作日內完成退款或換貨。  
   - 若退款，款項將退回原支付方式；若換貨，您將收到新商品並提供相同的運費方式。

若您在上述流程中有任何疑問，歡迎隨時撥打客服電話或直接回覆此郵件，我們將竭誠為您服務。  
再次對此次給您帶來的不便致歉，感謝您的耐心與配合。

祝 您一天順利！

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:中華民國,西元2025-10-18 15:30

**郵件品質評估（1–10 分）**  

| 評分項目 | 分數 | 評估說明 |
|---------|------|----------|
| 專業度 | 8 | 文字表達正式、條列清晰，但缺少公司標識、郵件標題與聯絡資訊，易造成信任度下降。 |
| 友善度 | 8 | 使用了道歉與感謝語，顯得貼心；但語氣略顯機械化，缺乏個人化情感。 |
| 完整度 | 7 | 步驟詳盡，流程完整；但重複內容（「感謝您的來信…」）與缺少重要資訊（公司地址、回覆期限、法律聲明）影響完整性。 |

---

## 具體改進建議

| 項目 | 問題 | 建議改進 |
|------|------|----------|
| **郵件標題** | 缺失 | 在主旨中直接說明郵件主題，例如「【退換貨協助】針對您的訂單（#123456）」 |
| **個人化稱呼** | 「王小明先生/女士